In [8]:
#%pip freeze -r requirements.txt

In [9]:
import os
import re
import requests
import qrcode
from PIL import Image, ImageDraw, ImageFont
from dotenv import load_dotenv
from fpdf import FPDF

In [10]:
# Load Spotify API credentials
load_dotenv()
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

In [11]:
# Constants
CARD_SIZE_PX = 1063  # 90mm x 90mm at 300 DPI
TEXT_IMG_DIR = "cards"
QR_IMG_DIR = "QR"
PDF_DIR = "pdfs"

In [12]:
# Get Spotify access token
def get_spotify_token():
    response = requests.post(
        "https://accounts.spotify.com/api/token",
        data={"grant_type": "client_credentials"},
        auth=(CLIENT_ID, CLIENT_SECRET),
    )
    response.raise_for_status()
    return response.json()["access_token"]

# Search for track and get URL + release year
def get_track_info(artist, title, token):
    query = f"{title} {artist}"
    headers = {"Authorization": f"Bearer {token}"}
    params = {"q": query, "type": "track", "limit": 1}
    response = requests.get("https://api.spotify.com/v1/search", headers=headers, params=params)
    items = response.json().get("tracks", {}).get("items", [])
    if not items:
        return None
    track = items[0]
    url = track["external_urls"]["spotify"]
    release_date = track["album"].get("release_date", "????")
    year = release_date.split("-")[0]
    return url, year

# Generate QR code image
def generate_qr_code(url, path):
    img = qrcode.make(url)
    img = img.resize((CARD_SIZE_PX, CARD_SIZE_PX), resample=Image.Resampling.LANCZOS)
    img.save(path)

# Generate text card image
def generate_text_card(title, artist, year, path, size=1063):
    img = Image.new("RGB", (size, size), "white")
    draw = ImageDraw.Draw(img)

    try:
        font = ImageFont.truetype("arial.ttf", 40)
    except IOError:
        font = ImageFont.load_default()

    text = f"\"{title}\"\nby {artist}\n({year})"
    spacing = 20
    align = "center"

    # Calculate text bounding box
    bbox = draw.multiline_textbbox((0, 0), text, font=font, spacing=spacing, align=align)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]

    # Center the text
    x = (size - text_width) // 2
    y = (size - text_height) // 2

    draw.multiline_text((x, y), text, font=font, fill="black", spacing=spacing, align=align)
    img.save(path)

# Combine both sides into 2-page PDF
def combine_to_pdf(text_img_path, qr_img_path, output_pdf_path):
    pdf = FPDF(unit="pt", format=[CARD_SIZE_PX, CARD_SIZE_PX])
    for img_path in [text_img_path, qr_img_path]:
        pdf.add_page()
        pdf.image(img_path, 0, 0, CARD_SIZE_PX, CARD_SIZE_PX)
    pdf.output(output_pdf_path)

# Read songs.txt
def parse_song_file(path):
    songs = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                parts = re.split(r"\s{2,}", line.strip())
                if len(parts) == 2:
                    artist, title = parts
                    songs.append((artist.strip(), title.strip()))
    return songs

# Ensure output directories exist
def ensure_dirs():
    os.makedirs(TEXT_IMG_DIR, exist_ok=True)
    os.makedirs(QR_IMG_DIR, exist_ok=True)
    os.makedirs(PDF_DIR, exist_ok=True)

# Main
def main():
    ensure_dirs()
    token = get_spotify_token()
    songs = parse_song_file("songs.txt")

    for artist, title in songs:
        print(f"🎵 Processing: {title} by {artist}")
        slug = f"{artist}_{title}".replace(" ", "_").replace("/", "_")

        info = get_track_info(artist, title, token)
        if not info:
            print("❌ Not found on Spotify")
            continue
        url, year = info

        text_img_path = os.path.join(TEXT_IMG_DIR, f"{slug}_text.png")
        qr_img_path = os.path.join(QR_IMG_DIR, f"{slug}_qr.png")
        pdf_path = os.path.join(PDF_DIR, f"{slug}_card.pdf")

        generate_text_card(title, artist, year, text_img_path)
        generate_qr_code(url, qr_img_path)
        combine_to_pdf(text_img_path, qr_img_path, pdf_path)
        print(f"✅ Created card: {pdf_path}\n")
        break
if __name__ == "__main__":
    main()

🎵 Processing: Paranoid by Black Sabbath
✅ Created card: pdfs/Black_Sabbath_Paranoid_card.pdf

